Mixture of Experts Notes prepared for [CAS ETH in ML in Finance and Insurance](https://finsuretech.ethz.ch/continuing-education/cas-ml-in-finance-and-insurance.html) by Eleni Verteouri.

Let´s start by training a Mixture of Experts (MoE) model: [HF-Demo-SwitchTransformers-summarization.ipynb](https://colab.research.google.com/drive/1aGGVHZmtKmcNBbAwa9hbu58DDpIuB5O4?usp=sharing#scrollTo=llowMkfHFC0Q)

# Prompt Engineering with Mixtral 8x7B

This guide provides some prompt examples demonstrating how to use Mixtral 8x7B and it's wide range of capabilities. [Source](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-mixtral-introduction.ipynb)

We will be using the official Python client from here: https://github.com/mistralai/client-python

Make sure to setup a `MISTRAL_API_KEY` before getting started with the guide. You can it here: https://console.mistral.ai/

In [ ]:
%%capture
!pip install mistralai

### Basic Usage

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv

load_dotenv()
import os

api_key = os.environ["MISTRAL_API_KEY"]
client = MistralClient(api_key=api_key)

In [ ]:
# helpful completion function

def get_completion(messages, model="mistral-small"):
    # No streaming
    chat_response = client.chat(
        model=model,
        messages=messages,
    )

    return chat_response


In [ ]:
messages = [
    ChatMessage(role="user", content="Tell me a joke about sharks")
]

chat_response = get_completion(messages)
print(chat_response)

id='2c7bed47132b45ce8a76ec8e2c5df25d' object='chat.completion' created=1706504806 model='mistral-small' choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Why do sharks swim in salt water? Because pepper would make them sneeze!'), finish_reason=<FinishReason.stop: 'stop'>)] usage=UsageInfo(prompt_tokens=15, total_tokens=32, completion_tokens=17)


In [ ]:
# print only the content
chat_response.choices[0].message.content

'Why do sharks swim in salt water? Because pepper would make them sneeze!'

### Using the Chat Template

To effectively prompt the Mistral 8x7B Instruct and get optimal outputs, it's recommended to use the following chat template:

```
<s>[INST] Instruction [/INST] Model answer</s>[INST] Follow-up instruction [/INST]
```

*Note that `<s>` and `</s>` are special tokens for beginning of string (BOS) and end of string (EOS) while [INST] and [/INST] are regular strings.*

In [ ]:
prompt = """[INST] You are a helpful code assistant. Your task is to generate a valid JSON object based on the given information:

name: John
lastname: Smith
address: #1 Samuel St.

Just generate the JSON object without explanations:
[/INST]"""

messages = [
    ChatMessage(role="user", content=prompt)
]

chat_response = get_completion(messages)
print(chat_response.choices[0].message.content)

{
"name": "John",
"lastname": "Smith",
"address": "#1 Samuel St."
}


Note the importance of the template that was used above. If we don't use the template, we get very different results. If we want to leverage the model capabilities in the proper way, we need to follow the format.

Here is another example that uses a conversation:

In [ ]:
prompt = """<s>[INST] What is your favorite condiment? [/INST]
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"</s> [INST] The right amount of what? [/INST]"""

messages = [
    ChatMessage(role="user", content=prompt)
]

chat_response = get_completion(messages)
print(chat_response.choices[0].message.content)

My apologies, I said "the right amount of zesty flavor." Fresh lemon juice can add a bright and tangy taste to various dishes, elevating their overall flavor profile.


We could also use the `ChatMessage` to define the different roles and content.

The example below shows a similar task in a multi-turn conversation:


In [ ]:
messages = [
    ChatMessage(role="user", content="What is your favorite condiment?"),
    ChatMessage(role="assistant", content="Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"),
    ChatMessage(role="user", content="The right amount of what?"),
]

chat_response = get_completion(messages)
print(chat_response.choices[0].message.content)

My apologies for any confusion. I meant to say that lemon juice adds a zesty flavour, which is a tangy and slightly sweet taste. It's a delightful addition to many dishes, in my humble opinion.


And here is the JSON object generation example from above using the `system`, `user`, and `assistant` roles.

In [ ]:
messages = [
    ChatMessage(role="system", content="You are a helpful code assistant. Your task is to generate a valid JSON object based on the given information."),
    ChatMessage(role="user", content="\n name: John\n lastname: Smith\n address: #1 Samuel St.\n would be converted to: "),
    ChatMessage(role="assistant", content="{\n \"address\": \"#1 Samuel St.\",\n \"lastname\": \"Smith\",\n \"name\": \"John\"\n}"),
    ChatMessage(role="user", content="name: Ted\n lastname: Pot\n address: #1 Bisson St.")
]

chat_response = get_completion(messages)
print(chat_response.choices[0].message.content)

{
 "address": "#1 Bisson St.",
 "lastname": "Pot",
 "name": "Ted"
}


### Code Generation

In [ ]:
messages = [
    ChatMessage(role="system", content="You are a helpful code assistant that help with writing Python code for a user requests. Please only produce the function and avoid explaining."),
    ChatMessage(role="user", content="Create a Python function to convert Celsius to Fahrenheit.")
]

chat_response = get_completion(messages)
print(chat_response.choices[0].message.content)

```python
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32
```


In [ ]:
# helpful completion function
def get_completion_safe(messages, model="mistral-small"):
    # No streaming
    chat_response = client.chat(
        model=model,
        messages=messages,
        safe_mode=True
    )

    return chat_response

messages = [
    ChatMessage(role="user", content="Say something very horrible and mean")
]

chat_response = get_completion(messages)
print(chat_response.choices[0].message.content)

I'm sorry, but I cannot comply with your request to say something horrible and mean. My purpose is to provide helpful, respectful, and positive interactions. It's important to treat everyone with kindness and respect, even in hypothetical situations.
